# Pre-process all the data

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [59]:
# Corpus reader:
import numpy as np
import random

import os
root = './Confs_newline/Conf2/'
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
reader = CategorizedPlaintextCorpusReader(root, r'.*\.txt', cat_pattern=r'(\w+)/*', encoding='utf-8')

In [60]:
print(reader.categories())
print(reader.fileids())

['kiz', 'kork', 'mutlu', 'notr', 'uzul']
['kiz.txt', 'kork.txt', 'mutlu.txt', 'notr.txt', 'uzul.txt']


In [61]:
### First, tokenize Punctuation: 
# create a token dictionary:
punc_dict= {'.':'||PERIOD||', ',': '||COMMA||', '"': '||QUOTATION_MARK||', ';': '||SEMICOLON||',
                '!': '||EXCLAMATION_MARK||', '?': '||QUESTION_MARK||', '(': '||LEFT_PAREN||',
                ')': '||RIGHT_PAREN||', '?': '||QUESTION_MARK||', 
                '\n': '||NEW_LINE||', '-': '||DASH||'}

In [62]:
def sent_tokenize_whole_tweets(text): # raw text --> whole tweets file content
    for key, token in punc_dict.items():
        text = text.replace(key, ' {} '.format(token))

    sentences= []
    for line in text.split('||NEW_LINE||'):
        line= line.strip()
        sentences.append(line)
    return sentences

In [63]:
all_text=[]
labels= []

In [64]:
for label,file_name in zip(reader.categories(), reader.fileids()):
    sentences= sent_tokenize_whole_tweets(reader.raw(file_name)) # --> this should return a list of contents
    labels.extend([label for i in sentences])
    all_text.extend([i.lower() for i in sentences])
print(len(labels))
print(len(all_text))
# Now, we have all tweets in all_text list!

3317
3317


## Transforming Text into Numbers

In [65]:
word_counts={}
for i in range(len(all_text)):
    for word in all_text[i].split(" "):
        word_counts[word] = word_counts.get(word,0) +1

vocab = set(word_counts.keys())
vocab_size = len(vocab)
print("Number of unique words: {} ".format(vocab_size))

sorted_word_counts= sorted(word_counts, key= word_counts.get, reverse= True)

int_to_vocab= {ii: word for ii,word in enumerate(sorted_word_counts)}
vocab_to_int= {word: ii for ii, word in int_to_vocab.items()}


Number of unique words: 3704 


In [69]:
all_text[2]

'hayır çok sev dizi yani aşırı kız'

In [67]:
int_to_vocab[0]

'çok'

In [32]:
word_counts['çok']

696

# Reducing Noise in the Input Data

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [39]:
vocab_to_int['her']

47

In [70]:
int_words = [vocab_to_int[word] for word in vocab]
print(int_words[:5])

[6, 1297, 1931, 1407, 2910]


In [71]:
threshold = 1e-5

word_counts_intwords = {vocab_to_int[word]:count for word,count in word_counts.items()}
total_count = vocab_size
freqs = {word: count/total_count for word, count in word_counts_intwords.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts_intwords}
# discard some frequent words, according to the subsampling equation
# create a new list of words for training


train_words = [word for word in word_counts_intwords if random.random() < (1 - p_drop[word])]
print(train_words[:30])

[263, 1457, 2022, 360, 2023, 496, 1007, 2028, 443, 322, 2029, 1008, 537, 538, 1462, 539, 2037, 2039, 310, 403, 774, 344, 1477, 1479, 2061, 506, 2068, 778, 192, 2072]


In [ ]:
# while training the network, do not feed the words not in this list!